In [ ]:
import numpy as np


# **The Mathematical Derivation for a 3-Layer Network**

Let's assume a 3-layer architecture: $X \longrightarrow H_1 \longrightarrow H_2 \longrightarrow \text{Output}$


### **1. Forward Pass Equations**

* **Layer 1**: $Z^{(1)} = W^{(1)} A^{(0)} + b^{(1)}$ , where $\quad A^{(0)} = X$
* **Activation 1**: $A^{(1)} = \sigma\!\left(Z^{(1)}\right)$ (ReLU)
* **Layer 2**: $Z^{(2)} = W^{(2)} A^{(1)} + b^{(2)}$
* **Activation 2**: $A^{(2)} = \sigma\!\left(Z^{(2)}\right)$ (ReLU)
* **Layer 3 (Output)**: $Z^{(3)} = W^{(3)} A^{(2)} + b^{(3)}$
* **Final Prediction**: $\hat{y} = \operatorname{Softmax}\!\left(Z^{(3)}\right)$

### **2. The Loss Function (Cross-Entropy)**

$$\mathcal{L}(y, \hat{y}) = -\sum_{i=1}^{K} y_i \log(\hat{y}_i)$$

### **3. Backpropagation (Chain Rule)**

For the output layer, the derivative of the Loss with respect to the pre-activation  simplifies elegantly to:
$$\delta^{(3)} = \frac{\partial \mathcal{L}}{\partial Z^{(3)}} = \hat{y} - y$$

Now, we propagate this error backward to find the gradients for the weights:

**For Layer 3:**

* $\frac{\partial \mathcal{L}}{\partial W^{(3)}} = \delta^{(3)} \left(A^{(2)}\right)^T$
* $\frac{\partial \mathcal{L}}{\partial b^{(3)}} = \delta^{(3)}$

**For Layer 2 (Hidden):**

* $\delta^{(2)} = \left(W^{(3)}\right)^T \delta^{(3)} \odot \sigma'\!\left(Z^{(2)}\right)$
* $\frac{\partial \mathcal{L}}{\partial W^{(2)}} = \delta^{(2)} \left(A^{(1)}\right)^T$
* $\frac{\partial \mathcal{L}}{\partial b^{(2)}} = \delta^{(2)}$

**For Layer 1 (Hidden):**

* $\delta^{(1)} = \left(W^{(2)}\right)^T \delta^{(2)} \odot \sigma'\!\left(Z^{(1)}\right)$
* $\frac{\partial \mathcal{L}}{\partial W^{(1)}} = \delta^{(1)} \left(A^{(0)}\right)^T$
* $\frac{\partial \mathcal{L}}{\partial b^{(1)}} = \delta^{(1)}$

> **Note**: The symbol $\odot$ represents the **Hadamard Product** (element-wise multiplication). This is crucial because each neuron’s error is gated by the derivative of its own activation function.


In [ ]:
class ScratchNet:
    def __init__(self, layers=[3072, 256, 128, 10]):
        self.params = {}
        # He Initialization for all layers
        for i in range(1, len(layers)):
            self.params[f'W{i}'] = np.random.randn(layers[i], layers[i-1]) * np.sqrt(2./layers[i-1])
            self.params[f'b{i}'] = np.zeros((layers[i], 1))

    def relu(self, Z):
        return np.maximum(0, Z)

    def relu_deriv(self, Z):
        return Z > 0

    def softmax(self, Z):
        # Numeric stability: subtract max(Z)
        expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))
        return expZ / np.sum(expZ, axis=0, keepdims=True)

    def forward(self, X):
        # Layer 1
        self.Z1 = np.dot(self.params['W1'], X) + self.params['b1']
        self.A1 = self.relu(self.Z1)
        # Layer 2
        self.Z2 = np.dot(self.params['W2'], self.A1) + self.params['b2']
        self.A2 = self.relu(self.Z2)
        # Layer 3 (Output)
        self.Z3 = np.dot(self.params['W3'], self.A2) + self.params['b3']
        self.A3 = self.softmax(self.Z3)
        return self.A3

    def backward(self, X, Y, A3, lr):
        m = X.shape[1]
        grads = {}
        
        # Output Error (Softmax + Cross-Entropy derivative)
        dZ3 = A3 - Y
        grads['dW3'] = (1/m) * np.dot(dZ3, self.A2.T)
        grads['db3'] = (1/m) * np.sum(dZ3, axis=1, keepdims=True)

        # Layer 2 Error
        dA2 = np.dot(self.params['W3'].T, dZ3)
        dZ2 = dA2 * self.relu_deriv(self.Z2)
        grads['dW2'] = (1/m) * np.dot(dZ2, self.A1.T)
        grads['db2'] = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

        # Layer 1 Error
        dA1 = np.dot(self.params['W2'].T, dZ2)
        dZ1 = dA1 * self.relu_deriv(self.Z1)
        grads['dW1'] = (1/m) * np.dot(dZ1, X.T)
        grads['db1'] = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

        # SGD Update
        for i in range(1, 4):
            self.params[f'W{i}'] -= lr * grads[f'dW{i}']
            self.params[f'b{i}'] -= lr * grads[f'db{i}']